In [ ]:
(ql:quickload :fep)

In [ ]:
(leap:setup-amber-paths)
(leap:source "leaprc.water.tip3p")
(leap:load-off "solvents.lib")
(leap:load-off "atomic_ions.lib")
(leap:load-atom-type-rules "atomtype_gff.def")
(leap:source "leaprc.ff14sb.redq")
(leap:source "leaprc.gaff")

In [ ]:
*default-pathname-defaults*

In [ ]:
(setf *default-pathname-defaults* (merge-pathnames "data/amber-dynamics/" *default-pathname-defaults*))

In [ ]:
(common-lisp:defparameter fep::*feps* (fep:load-feps "postcharge/feps.cando"))

In [ ]:
(common-lisp:defparameter fep::*receptor*
  (common-lisp:first (fep:receptors fep::*feps*)))

In [ ]:
(common-lisp:defparameter fep::*side-name* :complex)

In [ ]:
(common-lisp:defparameter fep::*morph*
  (fep::find-morph-with-name "AA-AB" fep::*feps*))

In [ ]:
(common-lisp:defparameter fep::*target* (fep:target fep::*morph*))

In [ ]:
(common-lisp:defparameter fep::*system*
  (cando:combine (chem:matter-copy (fep::molecule fep::*target*))
                 (chem:matter-copy fep::*receptor*)))

In [ ]:
(leap:solvate-box fep::*system*
                  (leap.core:lookup-variable (fep::solvent-box fep::*feps*))
                  (fep::solvent-buffer fep::*feps*)
                  (fep::solvent-closeness fep::*feps*))
(leap.add-ions:add-ions fep::*system* :|Cl-| 0)
(cando:save-mol2 fep::*system* (common-lisp:ensure-directories-exist "complex.mol2"))

In [ ]:
(leap.topology:save-amber-parm-format fep::*system* "complex.parm7" "complex.rst7")

In [ ]:
(ql:quickload :amber)

In [ ]:
*default-pathname-defaults*

In [ ]:
(core:copy-file "complex.parm7" (ensure-directories-exist "jobs/complex.parm7"))
(core:copy-file "complex.rst7" "jobs/complex.rst7")

In [ ]:
(let* ((top #P"complex.parm7")
           (crd #P"complex.rst7")
           (start (amber:simple-jupyter-job :topology-file top :coordinate-file crd))
           (min (amber:minimize :previous-job start))
           (heat (amber:heat :previous-job min))
           (press (amber:pressurize :previous-job heat))
           (dynamics (amber:dynamics :previous-job press :nstlim 100000 :ntwx 1000)))
      (amber:generate-all-code start
                               (list (amber:mdcrd dynamics))
                               :pathname-defaults #P"jobs/")
   (defparameter *start* start))

In [ ]:
(with-open-file (sout "/tmp/simple.dot" :direction :output)
    (graphviz-amber:draw-graph-stream *start* sout))

# Now go into the jobs directory and run the makefile

In [ ]:
(:= *m* (load-mol2 "complex.mol2"))

In [ ]:
(ql:quickload "netcdf")

In [ ]:
(ql:quickload :static-vectors)

In [ ]:
(ql:quickload :cando-jupyter)

In [ ]:
(:= *n* (netcdf::nc-open (probe-file "jobs/dynamics.nc") :mode netcdf-cffi:+nowrite+))

In [ ]:
(:= *agg* (load-mol2 "complex.mol2"))

In [ ]:
(:= *nc* (cando-jupyter:make-amber-netcdf-trajectory :netcdf *n* :matter *agg*))

In [ ]:
(:= *v* (show-amber-netcdf-trajectory *nc*))

In [ ]:
*v*

In [ ]:
(nglv::center *v*)

In [ ]:
(nglv::clear-representations *v*)

In [ ]:
(nglv::add-representation *v* "ball+stick" :selection "protein")

In [ ]:
(nglv::add-representation *v* "ribbon" :selection "protein")